# Import libraries

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, BatchNormalization, Dropout, Input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras
from datetime import datetime
from sklearn import preprocessing

# Load training dataset

In [ ]:
train_dataframe = pd.read_csv('dataset/1. istanbul/train_data.csv')

# Visualize some training data

In [ ]:
train_dataframe.head()

# Prepare training data

In [ ]:
train_x = train_dataframe.drop(['Subject ID', 'Class information', 'UPDRS'], axis=1)
train_x = train_x.as_matrix()
print(train_x.shape)

# Prepare training labels

In [ ]:
train_y = train_dataframe['Class information']
train_y = train_y.as_matrix()
print(train_y.shape)

# Prepare test data

In [ ]:
test_dataframe = pd.read_csv('dataset/1. istanbul/test_data.csv')
test_x = test_dataframe.drop(['Subject ID', 'Class information'], axis=1)
test_x = test_x.as_matrix()
print(test_x.shape)

# Prepare test labels

In [ ]:
test_y = test_dataframe['Class information']
test_y = test_y.as_matrix()
print(test_y.shape)

# Normalize data

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
test_x = min_max_scaler.fit_transform(test_x)
pd.DataFrame(train_x).head()

# Load first autoencoder model

In [ ]:
model = keras.models.load_model('autoencoder-model/2018-02-22-18-54-22/weights.hdf5')
model.summary()
SVG(model_to_dot(model, show_layer_names=False, show_shapes=True).create(prog='dot', format='svg'))

# Pass the inputs through first autoencoder

In [ ]:
train_x_after_autoencoder_1 = model.predict(train_x)
test_x_after_autoencoder_1 = model.predict(test_x)
pd.DataFrame(train_x_after_autoencoder_1).head()

# Create autoencoder 2

In [ ]:
inputs = Input(shape=(train_x_after_autoencoder_1.shape[1],))
outputs = Dense(10, activation='relu')(inputs)
outputs = Dense(7, activation='relu')(outputs)
outputs = Dense(10, activation='relu')(outputs)
outputs = Dense(train_x_after_autoencoder_1.shape[1], activation='sigmoid')(outputs)
model = Model(inputs, outputs)

model.summary()
SVG(model_to_dot(model, show_layer_names=False, show_shapes=True).create(prog='dot', format='svg'))

# Prepare callbacks

In [ ]:
# Tensorboard
date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir='Tensorboard2/' + date + '/')

# Model weight save
model_path = 'autoencoder-model/' + date
if not os.path.exists(model_path):
    os.makedirs(model_path)
check_pointer = ModelCheckpoint(filepath=model_path + '/weights.hdf5', verbose=1, save_best_only=True)

callbacks = [check_pointer, tensorboard_callback];

# Train the model

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.fit(train_x_after_autoencoder_1, train_x_after_autoencoder_1, validation_data=(test_x_after_autoencoder_1, test_x_after_autoencoder_1), epochs=6000, batch_size=20, callbacks=callbacks)

# The End